In [1]:
import numpy as np
import scipy.io
import glob
import os
import matplotlib.pyplot as plt
from scipy.signal import welch
import re
from tqdm import tqdm  # Para a barra de progresso
import matplotlib.ticker as ticker  # Para formatação dos rótulos dos eixos

# Diretório onde os arquivos .mat estão localizados
data_dir = 'C:/Users/FilipeAraujoXimenes/Documents/TG1/TG2/Data'

# Listar todos os arquivos .mat
filenames = glob.glob(os.path.join(data_dir, '*.mat'))

# Função para extrair o tempo do nome do arquivo
def extract_time(filename):
    base = os.path.basename(filename)
    # Procurar por 'qfour' seguido por números e possível ponto decimal
    match = re.match(r'qfour([\d\.]+)\.mat$', base)
    if match:
        return float(match.group(1))
    else:
        # Retornar float('inf') para arquivos que não correspondem ao padrão
        return float('inf')

# Ordenar os arquivos com base no tempo extraído
filenames = sorted(filenames, key=extract_time)

# Remover quaisquer arquivos que não conseguiram ser processados corretamente
filenames = [f for f in filenames if extract_time(f) != float('inf')]

# Atualizar N após a filtragem
N = len(filenames)

# Verificar se há arquivos para processar
if N == 0:
    raise ValueError("Nenhum arquivo .mat encontrado no diretório ou arquivos não seguem o padrão 'qfour<num>.mat'.")

# Carregar o primeiro arquivo para obter x, y, z
mat = scipy.io.loadmat(filenames[0])
x = mat['x'].flatten()
y = mat['y'].flatten()
z = mat['z'].flatten()

dx = x[1] - x[0]
Lx = x[-1] + dx
dz = z[1] - z[0]
Lz = z[-1] + dz

dalpha = 2 * np.pi / Lx
dbeta = 2 * np.pi / Lz
alphas = np.arange(0, 5) * dalpha
betas = np.arange(0, 11) * dbeta

# Evitar divisão por zero
epsilon = np.finfo(float).eps
alphas[0] = epsilon
betas[0] = epsilon

lambdax = 2 * np.pi / alphas
lambdaz = 2 * np.pi / betas

Retau = 180

lambdaxp = Retau * lambdax
lambdazp = Retau * lambdaz

lxptarget = 1000
lzptarget = 100

# Encontrar as posições mais próximas dos comprimentos de onda alvo
posa = np.argmin(np.abs(lambdaxp - lxptarget))
posb = np.argmin(np.abs(lambdazp - lzptarget))

ny = len(y)
ts = np.zeros(N)
X = np.zeros((3 * ny, N), dtype=complex)

# Loop para construir a matriz de dados X com tratamento de exceções
for i, filename in enumerate(tqdm(filenames, desc="Processando arquivos .mat")):
    try:
        ts[i] = extract_time(filename)
        mat = scipy.io.loadmat(filename)
        qfour = mat['qfour']
        
        # Verificação para garantir que o formato dos dados está correto
        if qfour.shape[0] <= posa or qfour.shape[2] <= posb:
            print(f"Dimensões incorretas no arquivo: {filename}, ignorando este arquivo.")
            continue
        
        X[:, i] = np.concatenate([
            qfour[posa, :, posb, 0].flatten(),  # u
            qfour[posa, :, posb, 1].flatten(),  # v
            qfour[posa, :, posb, 2].flatten()   # w
        ])
    
    except KeyError as e:
        print(f"Chave não encontrada no arquivo {filename}: {e}, ignorando este arquivo.")
    except Exception as e:
        print(f"Erro ao processar o arquivo {filename}: {e}, ignorando este arquivo.")

# Ordenar os dados por tempo
index = np.argsort(ts)
ts = ts[index]
X = X[:, index]

print(f"Número de arquivos processados: {N}")
print(f"Tamanho de ts: {ts.shape}")

# Função para ajustar os valores para a escala logarítmica base 10
def log_adjust_base10(data, threshold=1e-30):  # Threshold ajustado para valores muito pequenos
    data_adjusted = np.copy(data)
    data_adjusted[data_adjusted < threshold] = threshold
    return np.log10(data_adjusted)  # Log base 10 normal

# Função para configurar o eixo y sem notação científica
def format_labels(ax):
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

# Função para plotar os autovalores com tratamento de negativos e zero
def plot_autovalores(D, epsilon, title):
    # Ajustar os autovalores positivos para log
    D_pos = D.copy()
    D_pos[D < epsilon] = epsilon  # Substituir valores negativos e próximos de zero por epsilon

    # Plotar autovalores negativos em módulo
    if np.any(D < 0):
        D_neg = -D[D < 0]  # Inverter o sinal dos negativos para plotar em módulo
        
        # Primeiro gráfico com valores negativos em módulo
        fig, ax = plt.subplots()
        ax.plot(np.arange(len(D_neg)), D_neg, 'ro-', label='Autovalores Negativos (Módulo)')
        ax.set_xlabel('Índice do Autovalor')
        ax.set_ylabel('Valor Absoluto dos Autovalores Negativos')
        ax.set_title('Autovalores Negativos em Módulo')
        format_labels(ax)  # Aplicar formatação de rótulos
        plt.legend()
        plt.savefig('autovalores_negativos_modulo.png')
        plt.close()

        # Agora em log base 10
        D_neg_log10 = log_adjust_base10(D_neg)
        fig, ax = plt.subplots()
        ax.plot(np.arange(len(D_neg_log10)), D_neg_log10, 'ro-', label='Autovalores Negativos (Log Base 10)')
        ax.set_xlabel('Índice do Autovalor')
        ax.set_ylabel('Log Base 10 dos Autovalores Negativos')
        ax.set_title('Autovalores Negativos em Log Base 10')

        # Ajustar limites do eixo y para melhor visualização
        y_min, y_max = np.min(D_neg_log10), np.max(D_neg_log10)
        ax.set_ylim([y_min - 0.1, y_max + 0.1])  # Margem pequena para ajuste visual
        format_labels(ax)  # Aplicar formatação de rótulos
        plt.legend()
        plt.savefig('autovalores_negativos_log10.png')
        plt.close()

    # Criar figura para autovalores positivos e zero
    fig, ax = plt.subplots()
    ax.semilogy(np.arange(len(D_pos)), D_pos, 'b-', label='Autovalores Positivos ou Zero (Ajustados)')
    ax.set_xlabel('Índice do Autovalor')
    ax.set_ylabel('Valor do Autovalor (log)')
    ax.set_title(title)
    format_labels(ax)  # Aplicar formatação de rótulos
    plt.legend()
    plt.savefig('autovalores_positivos_ajustados.png')
    plt.close()

# Ordenar os dados por tempo
index = np.argsort(ts)
ts = ts[index]
X = X[:, index]

# Plotar dados
plt.figure(101)
plt.plot(ts)
plt.grid(True)
plt.title('Tempo ts')
plt.savefig('tempo_ts.png')
plt.close()

# Plotar cada uma das 5 componentes reais de X em figuras diferentes, a partir da figura 102
for i in range(5):
    plt.figure(102 + i)  # Começa a numeração das figuras a partir de 102
    plt.plot(ts, X[i, :].real.T)
    plt.grid(True)
    plt.title(f'Componente real do elemento {i + 1} de X')
    plt.xlabel('Tempo (ts)')
    plt.ylabel(f'Componente real {i + 1}')
    plt.savefig(f'componente_real_elemento_{i + 1}.png')
    plt.close()

# Preparar dados
dT = ts[-1] - ts[0]
dt = np.diff(ts, append=ts[-1] + (ts[-1] - ts[-2])) / dT
iavg = True
if iavg:
    Xavg = X @ dt
else:
    Xavg = np.zeros(X.shape[0], dtype=complex)

# Calcular os pesos espaciais usando a regra do trapézio
weights = -np.concatenate([
    [y[1] - y[0]],
    0.5 * (y[2:] - y[:-2]),
    [y[-1] - y[-2]]
])

print("Soma dos pesos temporais (dt):", np.sum(dt))
print("Soma dos pesos espaciais (weights):", np.sum(weights))

# Criar matriz diagonal de pesos espaciais
W_diag = np.concatenate([weights, weights, weights])
W = np.diag(W_diag)

# Calcular modos POD com conjugada transposta
sqrt_W = np.sqrt(W)
sqrt_dt = np.sqrt(dt)
aux = sqrt_W @ (X - Xavg[:, np.newaxis]) @ np.diag(sqrt_dt)
C = (aux @ aux.conj().T) / (N - 1)

# Verificar se C é Hermitiana
is_hermitian = np.allclose(C, C.conj().T)
print("A matriz C é Hermitiana:", is_hermitian)

# Obtém o valor do erro de máquina para o tipo float
epsilon = np.finfo(float).eps
print(f"Erro de máquina (epsilon): {epsilon}")

# Calcular autovalores e autovetores
D, U = np.linalg.eigh(C)

# Ordenar em ordem decrescente
idx = np.argsort(D)[::-1]
D = D[idx]
U = U[:, idx]

# Verificar autovalores negativos e autovalores próximos de zero
num_negatives = np.sum(D < 0)
num_near_zero = np.sum(np.abs(D) < epsilon)
print(f"Número de autovalores negativos: {num_negatives}")
print(f"Número de autovalores próximos de zero (|D| < epsilon): {num_near_zero}")

# Plotar autovalores ajustados (positivos e zero) e negativos separadamente
plot_autovalores(D, epsilon, title='Autovalores Positivos e Negativos Ajustados')

# Zerar autovalores negativos e próximos de zero para evitar problemas de instabilidade
D[D < epsilon] = epsilon

# Normalizar autovetores
for i in range(len(D)):
    U[:, i] = U[:, i] / np.sqrt(U[:, i].conj().T @ U[:, i])

phi = np.diag(1.0 / np.sqrt(np.diag(W))) @ U

# Projetar modos
coefs = np.linalg.solve(phi.conj().T @ W @ phi, phi.conj().T @ W @ (X - Xavg[:, np.newaxis]))

cond_number = np.linalg.cond(C)
print(f"Número de condição da matriz C: {cond_number}")

cumulative_energy = np.cumsum(D) / np.sum(D)
plt.figure()
plt.plot(cumulative_energy)
plt.xlabel('Número de Modos')
plt.ylabel('Energia Cumulativa')
plt.title('Energia Cumulativa dos Autovalores')
plt.grid(True)
plt.savefig('energia_cumulativa_autovalores.png')
plt.close()

# Generate and print cumulative energy percentages for each mode

# Calculate cumulative energy in percentage form for each mode
cumulative_energy_percentage = (np.cumsum(D) / np.sum(D)) * 100

# Print each mode's cumulative energy percentage
for mode, energy in enumerate(cumulative_energy_percentage, start=1):
    print(f"With {mode} mode(s), you have {energy:.8f}% of the total energy.")

# Supondo que as variáveis D, phi, coefs, X, Xavg, Xr, ts, y, ny, Retau já estejam definidas

# 1. Plotar autovalores
plt.figure(103, figsize=(12, 6))
plt.clf()

plt.subplot(1, 2, 1)
plt.plot(D)
plt.yscale('log')
plt.grid(True, which='both')
plt.title('Autovalores (D)')
plt.xlabel('Índice do Autovalor')
plt.ylabel('Valor do Autovalor')

plt.subplot(1, 2, 2)
plt.plot(1 - np.cumsum(D) / np.sum(D))
plt.yscale('log')
plt.grid(True, which='both')
plt.title('Energia Residual Cumulativa')
plt.xlabel('Número de Modos')
plt.ylabel('Energia Residual')

plt.suptitle(f"Valor mínimo de D: {np.min(D)}")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('autovalores_e_energia_residual.png')
plt.close()

# 2. Reconstrução e cálculo do erro
err = []
nmodes_list = np.arange(10, coefs.shape[0] + 1, 10)
for nmodes in nmodes_list:
    Xr = Xavg[:, np.newaxis] + phi[:, :nmodes] @ coefs[:nmodes, :]
    max_err = 100 * np.max(np.abs(X - Xr)) / np.max(np.abs(X))
    fro_err = 100 * np.linalg.norm(X - Xr, 'fro') / np.linalg.norm(X, 'fro')
    err.append([max_err, fro_err])

err = np.array(err).T

plt.figure(104, figsize=(8, 6))
plt.clf()
plt.plot(nmodes_list, err[0, :], label='L_inf')
plt.plot(nmodes_list, err[1, :], label='L2')
plt.yscale('log')
plt.grid(True, which='both')
plt.legend()
plt.xlabel('Número de Modos')
plt.ylabel('Erro de Reconstrução (%)')
plt.title('Erro de Reconstrução em Função do Número de Modos')
plt.tight_layout()
plt.savefig('erro_reconstrucao_numero_modos.png')
plt.close()

nmodes = 100
Xr = Xavg[:, np.newaxis] + phi[:, :nmodes] @ coefs[:nmodes, :]

# 3. Analisar formas dos modos
plt.figure(105, figsize=(18, 6))
plt.clf()

plt.subplot(1, 3, 1)
for i in range(6):
    plt.plot(np.abs(phi[:ny, i]), y, label=f'Modo #{i+1}')
plt.grid(True)
plt.legend(loc='center right')
plt.xlabel('|u|')
plt.ylabel('y')
plt.title('Componentes u dos modos')

plt.subplot(1, 3, 2)
for i in range(6):
    plt.plot(np.abs(phi[ny:2*ny, i]), y)
plt.grid(True)
plt.xlabel('|v|')
plt.ylabel('y')
plt.title('Componentes v dos modos')

plt.subplot(1, 3, 3)
for i in range(6):
    plt.plot(np.abs(phi[2*ny:, i]), y)
plt.grid(True)
plt.xlabel('|w|')
plt.ylabel('y')
plt.title('Componentes w dos modos')

plt.tight_layout()
plt.savefig('formas_modos.png')
plt.close()

# 4. Analisar formas dos modos em escala logarítmica (y+)
plt.figure(106, figsize=(18, 6))
plt.clf()

plt.subplot(1, 3, 1)
for i in range(6):
    plt.plot(np.abs(phi[:ny, i]), (y + 1) * Retau, label=f'Modo #{i+1}')
plt.grid(True)
plt.legend(loc='lower right')
plt.ylim([1e0, Retau])
plt.yscale('log')
plt.xlabel('|u|')
plt.ylabel('y+')
plt.title('Componentes u dos modos (y+)')

plt.subplot(1, 3, 2)
for i in range(6):
    plt.plot(np.abs(phi[ny:2*ny, i]), (y + 1) * Retau)
plt.grid(True)
plt.ylim([1e0, Retau])
plt.yscale('log')
plt.xlabel('|v|')
plt.ylabel('y+')
plt.title('Componentes v dos modos (y+)')

plt.subplot(1, 3, 3)
for i in range(6):
    plt.plot(np.abs(phi[2*ny:, i]), (y + 1) * Retau)
plt.grid(True)
plt.ylim([1e0, Retau])
plt.yscale('log')
plt.xlabel('|w|')
plt.ylabel('y+')
plt.title('Componentes w dos modos (y+)')

plt.tight_layout()
plt.savefig('formas_modos_escala_log_yplus.png')
plt.close()

# 5. Analisar histórico dos coeficientes
plt.figure(107, figsize=(12, 8))
plt.clf()

for i in range(3):
    plt.subplot(3, 1, i + 1)
    ax1 = plt.gca()
    ax2 = ax1.twinx()
    ax1.plot(ts, coefs[i, :].real, 'b-', label='Real')
    ax2.plot(ts, coefs[i, :].imag, 'r--', label='Imag')
    ax1.set_ylabel('Parte Real')
    ax2.set_ylabel('Parte Imaginária')
    plt.title(f'Coeficiente {i+1}')
    ax1.grid(True)
    if i == 2:
        ax1.set_xlabel('Tempo ts')
    if i == 0:
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')
plt.tight_layout()
plt.savefig('historico_coeficientes.png')
plt.close()

# 6. Verificar precisão da reconstrução
step = 20  # Número de subplots

plt.figure(108, figsize=(16, 12))
plt.clf()

for i in range(step):
    plt.subplot(4, 5, i + 1)
    plt.plot(np.abs(X[:ny, i]), y, label='Original')
    plt.plot(np.abs(Xr[:ny, i]), y, label='Reconstruído', linestyle='--')
    plt.xlabel('|u|')
    plt.ylabel('y')
    plt.title(f't={ts[i]:.2f}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |u| Original e Reconstruído')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('comparacao_u_original_reconstruido.png')
plt.close()

plt.figure(109, figsize=(16, 12))
plt.clf()

for i in range(step):
    plt.subplot(4, 5, i + 1)
    plt.plot(np.abs(X[ny:2*ny, i]), y, label='Original')
    plt.plot(np.abs(Xr[ny:2*ny, i]), y, label='Reconstruído', linestyle='--')
    plt.xlabel('|v|')
    plt.ylabel('y')
    plt.title(f't={ts[i]:.2f}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |v| Original e Reconstruído')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('comparacao_v_original_reconstruido.png')
plt.close()

plt.figure(110, figsize=(16, 12))
plt.clf()

for i in range(step):
    plt.subplot(4, 5, i + 1)
    plt.plot(np.abs(X[2*ny:, i]), y, label='Original')
    plt.plot(np.abs(Xr[2*ny:, i]), y, label='Reconstruído', linestyle='--')
    plt.xlabel('|w|')
    plt.ylabel('y')
    plt.title(f't={ts[i]:.2f}')
    plt.grid(True)
    if i == 0:
        plt.legend()
plt.suptitle('Comparação de |w| Original e Reconstruído')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('comparacao_w_original_reconstruido.png')
plt.close()

# 7. Calcular espectros dos coeficientes (delta_t variável)
plt.figure(111, figsize=(16, 12))
plt.clf()

p = coefs[:, np.concatenate([np.arange(0, 4799, 2), np.arange(4800, coefs.shape[1])])]
delta_t = ts[4801] - ts[4800]
nfft = 2**6
fs = 1 / delta_t

for i in range(16):
    plt.subplot(4, 4, i + 1)
    f_real, Pxx_real = welch(p[i, :].real, fs=fs, nperseg=nfft, noverlap=nfft//2)
    f_imag, Pxx_imag = welch(p[i, :].imag, fs=fs, nperseg=nfft, noverlap=nfft//2)
    plt.plot(f_real, Pxx_real, 'b-', label='Real')
    plt.plot(f_imag, Pxx_imag, 'r--', label='Imag')
    plt.yscale('log')
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('PSD')
    plt.title(f'Modo {i+1}')
    plt.grid(True, which='both')
    if i == 0:
        plt.legend()
plt.suptitle('Espectros dos Coeficientes (delta_t variável)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('espectros_coeficientes_delta_t_variavel.png')
plt.close()

# 8. Calcular espectros dos coeficientes (delta_t constante)
plt.figure(112, figsize=(16, 12))
plt.clf()

p = coefs[:, :4800]
delta_t = ts[1] - ts[0]
nfft = 2**6
fs = 1 / delta_t

for i in range(16):
    plt.subplot(4, 4, i + 1)
    f_real, Pxx_real = welch(p[i, :].real, fs=fs, nperseg=nfft, noverlap=nfft//2)
    f_imag, Pxx_imag = welch(p[i, :].imag, fs=fs, nperseg=nfft, noverlap=nfft//2)
    plt.plot(f_real, Pxx_real, 'b-', label='Real')
    plt.plot(f_imag, Pxx_imag, 'r--', label='Imag')
    plt.yscale('log')
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('PSD')
    plt.title(f'Modo {i+1}')
    plt.grid(True, which='both')
    if i == 0:
        plt.legend()
plt.suptitle('Espectros dos Coeficientes (delta_t constante)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('espectros_coeficientes_delta_t_constante.png')
plt.close()

# Selecionar os coeficientes de 1 a 387
indices = np.arange(0, 387)
coeficientes = coefs.flatten()[indices]

# Plotar os coeficientes
plt.figure(figsize=(10, 6))
plt.plot(indices + 1, np.abs(coeficientes))
plt.xlabel('Índice')
plt.ylabel('Magnitude dos Coeficientes')
plt.title('Plot dos Coeficientes de 1 a 387')
plt.grid(True)
plt.savefig('coeficientes_1_a_387.png')
plt.close()

# Defina o número de modos que você deseja utilizar
nmodes = 387  # Escolha o número de modos desejado

# Reconstrução dos dados usando os primeiros nmodes modos
Xr = Xavg[:, np.newaxis] + phi[:, :nmodes] @ coefs[:nmodes, :]

# Salvar apenas os primeiros nmodes coeficientes
scipy.io.savemat('temp_coefs.mat', {'coefs': coefs[:nmodes, :], 'ts': ts})

# Salvar apenas os primeiros nmodes modos espaciais
scipy.io.savemat('spatial_modes.mat', {'phi': phi[:, :nmodes]})

# Salvar Xavg e y (sem alterações)
scipy.io.savemat('mean_flow.mat', {'Xavg': Xavg, 'y': y})

# (Opcional) Salvar o valor de nmodes para referência futura
scipy.io.savemat('parameters.mat', {'nmodes': nmodes})


Processando arquivos .mat: 100%|██████████| 10401/10401 [00:55<00:00, 187.83it/s]


Número de arquivos processados: 10401
Tamanho de ts: (10401,)
Soma dos pesos temporais (dt): 1.0001250000000004
Soma dos pesos espaciais (weights): 2.000301181303796
A matriz C é Hermitiana: True
Erro de máquina (epsilon): 2.220446049250313e-16
Número de autovalores negativos: 72
Número de autovalores próximos de zero (|D| < epsilon): 266
Número de condição da matriz C: 1.3060662330527177e+33
With 1 mode(s), you have 34.98241042% of the total energy.
With 2 mode(s), you have 67.81909534% of the total energy.
With 3 mode(s), you have 74.65402199% of the total energy.
With 4 mode(s), you have 80.87495164% of the total energy.
With 5 mode(s), you have 83.70097183% of the total energy.
With 6 mode(s), you have 86.26156804% of the total energy.
With 7 mode(s), you have 87.89943203% of the total energy.
With 8 mode(s), you have 89.41012556% of the total energy.
With 9 mode(s), you have 90.55593282% of the total energy.
With 10 mode(s), you have 91.60660287% of the total energy.
With 11 mode(